# Visualization of motion mapping  by iteration method in 2D toy example


In [ ]:
import numpy as np
import time
import matplotlib.pyplot as plt

# https://docs.scipy.org/doc/scipy/reference/optimize.html
from scipy.optimize import minimize

# load iteration mapping code
import iteration_mapping as iter
import data_generation as dg

# https://github.com/moble/quaternion  install quaternion and numba
import quaternion # [w,x,y,z] order


In [ ]:
# create grid_lines for visualization
downsampling = 10
nbGrid = 20 * downsampling + 1
def generate_grid(xk, xk2, offset = 1.5):
    grid_lines=[]
    x_grid_ = np.linspace(np.min(xk[:,0]) - offset, np.max(xk[:,0]) + offset, nbGrid)
    y_grid_ = np.linspace(np.min(xk[:,1]) - offset, np.max(xk[:,1]) + offset, nbGrid)
    x_grid, y_grid = np.meshgrid(x_grid_, y_grid_)
    for ii in range(nbGrid):
        for jj in range(nbGrid):               
            x_grid_point = np.array([x_grid[ii,jj],y_grid[ii,jj] ])
            grid_lines.append(x_grid_point)

    grid_lines = np.asarray(grid_lines)
    return grid_lines


## Plot all objects on two sides

In [ ]:
# set sampling points for each object
sp = 100 # 1,10,100 
obj = dg.Shape

# generate test objects
p0 = obj(sample_points=sp, type=0,translation=np.array([0.7,0.9]),scale=0.1)
p1 = obj(sample_points=sp, type=1,translation=np.array([0,1]),scale=0.15)
p2 = obj(sample_points=sp, type=2,translation=np.array([1,0]),scale=0.15)
p3 = obj(sample_points=sp, type=3,translation=np.array([1.2,0.5]),scale=0.15)
p4 = obj(sample_points=sp, type=4,translation=np.array([0,0]),scale=0.5)


x_data = np.concatenate([p0.xy,p1.xy,p2.xy,p3.xy,p4.xy],axis=0)
x_key = [p0.key_points,p1.key_points,p2.key_points,p3.key_points,p4.key_points]
x_data = np.concatenate([x_data, np.repeat([[1.,0,0,0]],5*sp,axis=0)],axis=1)
fig = plt.figure(figsize=(10,10./2))
ax = fig.add_subplot(111)
ax.set_aspect("equal")
ax.scatter(x_data[:,0], x_data[:,1],s=1)
xk = np.concatenate([p0.xy[0,None,:],p1.xy[0,None,:],p2.xy[0,None,:],p3.xy[0,None,:],p4.xy[0,None,:]],axis=0)
ax.set_title('Objects on the local side')
# print xk
fig = plt.figure(figsize=(10,10./2))

ax = fig.add_subplot( 111,sharey=ax)
# sharey=ax
ax.set_aspect("equal")

# set the rotation angle on objects
angle = [0,-30,60,30,30.]
p0 = obj(sample_points=sp, type=0,translation=np.array([0.7,0.7])+np.array([0.8,0.5]),theta=angle[0],scale=0.1)
p1 = obj(sample_points=sp, type=1,translation=np.array([0,1])+np.array([0.4,0.2]),theta=angle[1],scale=0.15)
p2 = obj(sample_points=sp, type=2,translation=np.array([1,0])+np.array([0.7,0.2]),theta=angle[2],scale=0.15)
p3 = obj(sample_points=sp, type=3,translation=np.array([1.5,0.8])+np.array([0.5,0.3]),theta=angle[3],scale=0.15)
p4 = obj(sample_points=sp, type=4,translation=np.array([0,0])+np.array([0,0.2]),theta=angle[4],scale=0.5)
xk2 = np.concatenate([p0.xy[0,None,:],p1.xy[0,None,:],p2.xy[0,None,:],p3.xy[0,None,:],p4.xy[0,None,:]],axis=0)
y_key = [p0.key_points,p1.key_points,p2.key_points,p3.key_points,p4.key_points]

p = [p0.xy,p1.xy,p2.xy,p3.xy,p4.xy]

q_all =np.zeros([5,4])
for i in range(5):
    q = quaternion.from_euler_angles([-angle[i]*np.pi/180.,0,0])
    q_tmp = quaternion.as_float_array(q)
    q_all[i,:] = q_tmp

    p[i] = np.concatenate([p[i],np.repeat(q_tmp.reshape(1,-1), sp,axis=0)],axis=1)
xk2 = np.concatenate([xk2, q_all],axis=1)
y_data = np.concatenate([p[0],p[1],p[2],p[3],p[4]],axis=0)
ax.scatter(y_data[:,0], y_data[:,1],s=1)
ax.set_title('Objects on the remote side')

## iteration algorithm with Gaussian RBF

In [ ]:
para = np.array([100, 0.6, 0.95])  # iteration algorithm parameter: [K, \mu, \beta]

## Data to be mapped:
# x_data: n x 6, [x, y, qw, qx, qy, qz] position and orientation of objects on the local side 
# y_data: n x 6, [x, y, qw, qx, qy, qz] position and orientation of objects on the remote side 

M = iter.iteration(para, x_data, y_data) # Training

## Visualization in 2D 

In [ ]:
grid_lines = generate_grid(xk, xk2)
print grid_lines.shape[0]

nbGrid =  int(np.sqrt(grid_lines.shape[0]))
tmp0 = grid_lines.reshape(nbGrid,nbGrid,2)
print tmp0.shape
x_grid = tmp0[:,:,0];y_grid = tmp0[:,:,1]

In [ ]:
def plot_2D_pos_ori(ax, data,color='r',alpha=1,length=0.01,head_width=0.007,lw=1,down_sampling=1,zorder =0, x_axis=True):
    # data: [n, 6]
    arrow_len = np.array([length, 0, 0])
    if data.ndim==1:
        data = data.reshape(-1,1)        
    for i in range(data.shape[0]):        
        x, y, q = data[i,0], data[i,1], quaternion.from_float_array(data[i,2:])
        if x_axis:
            new_arrow = quaternion.rotate_vectors(q, arrow_len) 
        else:
            new_arrow = quaternion.rotate_vectors(q * quaternion.from_euler_angles(np.pi/2,0,0), arrow_len) 

        tmp =  length* np.array([np.cos(data[i,2]), np.sin(data[i,2])])
#         print tmp
        ax.arrow(x, y, new_arrow[0],new_arrow[1],width=0.004,head_width=head_width,
                 color=color,alpha=alpha,lw=lw,zorder=zorder,length_includes_head=False)
    


In [ ]:
plot_list = []
for i in range(0,nbGrid,downsampling):
    plot_list += range(i*nbGrid, (i+1)*nbGrid,downsampling )

In [ ]:
%matplotlib inline
red_color = np.array([228,73,26,255])/255.
green_color = np.array([100,165,74,255])/255.
xk_color = np.array([[38, 70, 83,255],
                    [42, 157, 143,255],
                    [242, 220, 166,255],
                    [243, 155, 83,255],
                    [197, 61, 27,255]])/255.

In [ ]:
grid_lines = generate_grid(xk,xk2)

# add identity quaternion on grid poins of the local side
grid_lines = np.concatenate([grid_lines, np.repeat(np.array([[1.,0,0,0]]),nbGrid**2,axis=0 )   ],axis=1) 

# map grid points to the remote side
grid_lines2 = M.forward(grid_lines) 

# plot 
fig_width = 85/25.4 *8/9 # mm to inch
fig, ax = plt.subplots(ncols=2, figsize=(fig_width, fig_width/2 *4/5))
fig.subplots_adjust(left=0, bottom=0, right=1, top=1, wspace=0.33, hspace=0)
plt.rcParams["font.family"] = "Times New Roman"
plt.rcParams["font.size"] = "8"
# plt.rcParams["text.usetex"] = True
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42

grids1 = grid_lines[:,:].reshape(nbGrid,nbGrid,6)
grids2 = grid_lines2[:,:].reshape(nbGrid,nbGrid,6)

set_alpha = 0.5
for i in range(0,nbGrid,downsampling):
    ax[0].plot(grids1[i,:,0], grids1[i,:,1], c = 'darkgray', alpha=set_alpha,linewidth=0.5)
    ax[0].plot(grids1[:,i,0], grids1[:,i,1], c = 'darkgray', alpha=set_alpha,linewidth=0.5)
    ax[1].plot(grids2[i,:,0], grids2[i,:,1], c = 'darkgray', alpha=set_alpha,linewidth=0.5)
    ax[1].plot(grids2[:,i,0], grids2[:,i,1], c = 'darkgray', alpha=set_alpha,linewidth=0.5)

ax[0].axis('equal')
ax[1].axis('equal')

i=0
for data in x_key:
    ax[0].plot(data[:,0],data[:,1],c=xk_color[i,:],linewidth=1,zorder = 100)
    i = i+1
i=0
for data in y_key:
    ax[1].plot(data[:,0],data[:,1],c=xk_color[i,:],linewidth=1,zorder = 100)
    i = i+1

In [ ]:
%matplotlib inline
# fig, ax = plt.subplots(ncols=2, figsize=(fig_width, fig_width/2 *4/5))
# fig.subplots_adjust(left=0, bottom=0, right=1, top=1,
#                 wspace=0.33, hspace=0)
fig_width = 90/25.4 /2 # mm to inch

rate = 1.
ratio = 1
fig0 = plt.figure(figsize=(fig_width*rate, fig_width *rate * ratio))
ax0 = fig0.add_subplot(111)
fig1 = plt.figure(figsize=(fig_width, fig_width ))

# ax1 = fig1.add_subplot(111, sharey=ax0)
ax1 = fig1.add_subplot(111)

ax = [ax0,ax1]
set_alpha = 1
grid_color = 'lightgray'
for i in range(0, nbGrid, downsampling):
    ax[0].plot(grids1[i,:,0], grids1[i,:,1], c = grid_color, alpha=set_alpha,linewidth=0.5,zorder = 0)
    ax[0].plot(grids1[:,i,0], grids1[:,i,1], c = grid_color, alpha=set_alpha,linewidth=0.5,zorder = 0)
    ax[1].plot(grids2[i,:,0], grids2[i,:,1], c = grid_color, alpha=set_alpha,linewidth=0.5,zorder = 0)
    ax[1].plot(grids2[:,i,0], grids2[:,i,1], c = grid_color, alpha=set_alpha,linewidth=0.5,zorder = 0)
# Set the zorder for the artist. Artists with lower zorder values are drawn first.

plot_2D_pos_ori(ax[1],grid_lines2[plot_list,:],color='gray',alpha=set_alpha,lw=0.4,
                head_width=0.02,length=0.09,zorder = 5)
plot_2D_pos_ori(ax[0],grid_lines[plot_list,:],color='gray',alpha=set_alpha,lw=0.4,
                head_width=0.02,length=0.09,zorder = 5)
 
x_lim = np.array([-0.5,2.5])
y_lim = np.array([-0.5,1.75])+0.05  
ax[0].set_xlim(x_lim)
ax[0].set_ylim(y_lim)

# ax[0].set_aspect(1./ax[0].get_data_ratio())
ax[0].set_aspect(1.)
ax[1].set_aspect(1.)
ax[1].set_xlim(x_lim)
ax[1].set_ylim(y_lim)

ax[0].set_xticks([],())
ax[1].set_xticks([],())
# ax[0].set_yticks([],())
# ax[1].set_yticks([],())
i=0
for data in x_key:
    ax[0].plot(data[:,0],data[:,1],c=xk_color[i,:],linewidth=1,zorder = 100)
    i = i+1
i=0
for data in y_key:
    ax[1].plot(data[:,0],data[:,1],c=xk_color[i,:],linewidth=1,zorder = 100)
    i = i+1
# ax[0].set_aspect(1, adjustable='box')
# ax[1].set_aspect(1, adjustable='box')


# fig0.savefig('figures/2D_local_sp100.pdf',format='pdf',bbox_inches='tight',  pad_inches=0.0)
# fig1.savefig('figures/2D_remote_sp100.pdf',format='pdf',bbox_inches='tight',  pad_inches=0.0)
